<h1 align="center"><font size="6">Applied Data Science Capstone</font></h1>

This notebook is used to complete the capstone project of the Coursera course - Applied Data Science Capstone.

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
